# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU,CuDNNLSTM,CuDNNGRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

C:\Users\ndrs\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 74094599461318206
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9233114071
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10927477591585745678
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tok=Tokenizer()
    tok.fit_on_texts(x)
    
    return tok.texts_to_sequences(x), tok

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length==None :
        length=len(max(x,key=len))
    return pad_sequences(x,maxlen=length,padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [17]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    
    inp_seq =Input(input_shape[1:])
    print(inp_seq)
    rnn=CuDNNLSTM(256,return_sequences=True)(inp_seq)
    logits=TimeDistributed(Dense(french_vocab_size))(rnn)
    cost=Activation('softmax')(logits)
    model=Model(inp_seq,cost)
    learning_rate=1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
#ORIG BATCH SIZE=1024
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=32, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Tensor("input_13:0", shape=(?, 21, 1), dtype=float32)
Tensor("input_14:0", shape=(?, 21, 1), dtype=float32)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 21s 188us/step - loss: 1.2899 - acc: 0.6357 - val_loss: nan - val_acc: 0.6719
Epoch 2/10
110288/110288 [==============================] - 19s 175us/step - loss: 0.9610 - acc: 0.6974 - val_loss: nan - val_acc: 0.7260
Epoch 3/10
110288/110288 [==============================] - 19s 176us/step - loss: 0.8300 - acc: 0.7335 - val_loss: nan - val_acc: 0.7488
Epoch 4/10
110288/110288 [==============================] - 20s 179us/step - loss: 0.7651 - acc: 0.7494 - val_loss: nan - val_acc: 0.7667
Epoch 5/10
110288/110288 [==============================] - 20s 179us/step - loss: 0.7126 - acc: 0.7643 - val_loss: nan - val_acc: 0.7714
Epoch 6/10
110288/110288 [==============================] - 20s 182us/step - loss: 0.6759 - acc: 0.7753 - val_loss: nan - val_acc: 0.7813
Epoch 7/10
11

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [18]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    inp_seq =Input(input_shape[1:])
    print(inp_seq)
    embed=Embedding(input_dim=english_vocab_size,output_dim=64,input_length=output_sequence_length)(inp_seq)
    rnn=CuDNNLSTM(256,return_sequences=True)(embed)
    logits=TimeDistributed(Dense(french_vocab_size))(rnn)
    cost=Activation('softmax')(logits)
    model=Model(inp_seq,cost)
    learning_rate=1e-3
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(1e-3),metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network
rnn_embed_model = embed_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
# TODO: Train the neural network
rnn_embed_model.fit(tmp_x,preproc_french_sentences,epochs=10,batch_size=32,validation_split=0.2)
# TODO: Print prediction(s)
print(logits_to_text(rnn_embed_model.predict(tmp_x[:1])[0],french_tokenizer))

Tensor("input_15:0", shape=(?, 21), dtype=float32)
Tensor("input_16:0", shape=(?, 21), dtype=float32)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 23s 209us/step - loss: 0.7997 - acc: 0.7936 - val_loss: nan - val_acc: 0.8946
Epoch 2/10
110288/110288 [==============================] - 22s 195us/step - loss: 0.2754 - acc: 0.9080 - val_loss: nan - val_acc: 0.9183
Epoch 3/10
110288/110288 [==============================] - 22s 201us/step - loss: 0.2236 - acc: 0.9229 - val_loss: nan - val_acc: 0.9264
Epoch 4/10
110288/110288 [==============================] - 21s 195us/step - loss: 0.2015 - acc: 0.9291 - val_loss: nan - val_acc: 0.9281
Epoch 5/10
110288/110288 [==============================] - 20s 184us/step - loss: 0.1887 - acc: 0.9327 - val_loss: nan - val_acc: 0.9312
Epoch 6/10
110288/110288 [==============================] - 20s 184us/step - loss: 0.1811 - acc: 0.9347 - val_loss: nan - val_acc: 0.9342
Epoch 7/10
110288/1

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [19]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inp_seq=Input(input_shape[1:])
    rnn=Bidirectional(CuDNNLSTM(256,return_sequences=True))(inp_seq)
    logits=TimeDistributed(Dense(french_vocab_size))(rnn)
    cost=Activation('softmax')(logits)
    model=Model(inp_seq,cost)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(1e-3),metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)
#logic behind is to pad Train X and have rows equal to french max sequence length
tmp_x=pad(preproc_english_sentences,max_french_sequence_length)
tmp_x=tmp_x.reshape(-1,tmp_x.shape[1],1)

rnn_bd_model=bd_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
rnn_bd_model.fit(tmp_x,preproc_french_sentences,batch_size=32,epochs=10,validation_split=0.2)
# TODO: Train and Print prediction(s)
print(logits_to_text(rnn_bd_model.predict(tmp_x[:1])[0],french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 29s 266us/step - loss: 1.1414 - acc: 0.6646 - val_loss: nan - val_acc: 0.6983
Epoch 2/10
110288/110288 [==============================] - 28s 256us/step - loss: 0.8580 - acc: 0.7180 - val_loss: nan - val_acc: 0.7429
Epoch 3/10
110288/110288 [==============================] - 28s 252us/step - loss: 0.7342 - acc: 0.7524 - val_loss: nan - val_acc: 0.7574
Epoch 4/10
110288/110288 [==============================] - 27s 244us/step - loss: 0.6464 - acc: 0.7802 - val_loss: nan - val_acc: 0.7908
Epoch 5/10
110288/110288 [==============================] - 27s 248us/step - loss: 0.5955 - acc: 0.7966 - val_loss: nan - val_acc: 0.8109
Epoch 6/10
110288/110288 [==============================] - 27s 248us/step - loss: 0.5521 - acc: 0.8110 - val_loss: nan - val_acc: 0.8063
Epoch 7/10
110288/110288 [==============================] - 27s 241us/step - loss: 0.5222 - acc: 0.8210 - val_loss: nan -

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [16]:
#ONESHOT METHOD
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    enc_inp=Input(input_shape[1:])
    enc_rnn=CuDNNLSTM(256)(enc_inp)
    enc_vec=RepeatVector(output_sequence_length)(enc_rnn)
    dec_rnn=CuDNNLSTM(256,return_sequences=True)(enc_vec)
    logits=TimeDistributed(Dense(french_vocab_size))(dec_rnn)
    cost=Activation('softmax')(logits)
    
    model=Model(enc_inp,cost)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(1e-3),metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

tmp_x=pad(preproc_english_sentences,max_french_sequence_length)
tmp_x=tmp_x.reshape(-1,tmp_x.shape[1],1)

ecdc_model=encdec_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
ecdc_model.fit(tmp_x,preproc_french_sentences,batch_size=32,epochs=10,validation_split=0.2)
# OPTIONAL: Train and Print prediction(s)
print(logits_to_text(ecdc_model.predict(tmp_x[:1])[0],french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 29s 262us/step - loss: 1.4316 - acc: 0.6121 - val_loss: nan - val_acc: 0.6621
Epoch 2/10
110288/110288 [==============================] - 28s 256us/step - loss: 1.0342 - acc: 0.6766 - val_loss: nan - val_acc: 0.6904
Epoch 3/10
110288/110288 [==============================] - 28s 254us/step - loss: 0.8853 - acc: 0.7018 - val_loss: nan - val_acc: 0.7151
Epoch 4/10
110288/110288 [==============================] - 28s 255us/step - loss: 0.7373 - acc: 0.7428 - val_loss: nan - val_acc: 0.7724
Epoch 5/10
110288/110288 [==============================] - 28s 254us/step - loss: 0.6364 - acc: 0.7730 - val_loss: nan - val_acc: 0.7822
Epoch 6/10
110288/110288 [==============================] - 28s 252us/step - loss: 0.5597 - acc: 0.7943 - val_loss: nan - val_acc: 0.7849
Epoch 7/10
110288/110288 [==============================] - 29s 259us/step - loss: 0.5242 - acc: 0.8035 - val_loss: nan -

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [20]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inp_seq=Input(input_shape[1:])
    embed=Embedding(input_dim=english_vocab_size,output_dim=64,input_length=output_sequence_length)(inp_seq)
    enc_rnn=Bidirectional(CuDNNLSTM(256))(embed)
    enc_vec=RepeatVector(output_sequence_length)(enc_rnn)
    dec_rnn=Bidirectional(CuDNNLSTM(256,return_sequences=True))(enc_vec)
    logits=TimeDistributed(Dense(french_vocab_size,))(dec_rnn)
    cost=Activation('softmax')(logits)
    
    model=Model(inp_seq,cost)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(1e-3),metrics=['accuracy'])
    return model
tests.test_model_final(model_final)

tmp_x=pad(preproc_english_sentences,max_french_sequence_length)

rnn_model_final=model_final(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)


print('Final Model Loaded')
# TODO: Train the final model
rnn_model_final.fit(tmp_x,preproc_french_sentences,batch_size=32,epochs=10,validation_split=0.2)
print(logits_to_text(rnn_model_final.predict(tmp_x[:1])[0],french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 50s 456us/step - loss: 0.9241 - acc: 0.7441 - val_loss: nan - val_acc: 0.8724
Epoch 2/10
110288/110288 [==============================] - 46s 418us/step - loss: 0.2677 - acc: 0.9153 - val_loss: nan - val_acc: 0.9423
Epoch 3/10
110288/110288 [==============================] - 46s 415us/step - loss: 0.1587 - acc: 0.9477 - val_loss: nan - val_acc: 0.9553
Epoch 4/10
110288/110288 [==============================] - 47s 431us/step - loss: 0.1191 - acc: 0.9607 - val_loss: nan - val_acc: 0.9664
Epoch 5/10
110288/110288 [==============================] - 47s 426us/step - loss: 0.0934 - acc: 0.9695 - val_loss: nan - val_acc: 0.9633
Epoch 6/10
110288/110288 [==============================] - 47s 423us/step - loss: 0.0763 - acc: 0.9750 - val_loss: nan - val_acc: 0.9715
Epoch 7/10
110288/110288 [==============================] - 46s 420us/step - loss: 0.0646 - acc: 0.979

## Prediction (IMPLEMENTATION)

In [75]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    #tmp_x=pad(x,max_french_sequence_length)
   # print(tmp_x.shape)
    model=model_final(x.shape,y.shape[1],len(x_tk.word_index),len(y_tk.word_index))
    model.fit(x,y,batch_size=32,epochs=10,validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 45s 412us/step - loss: 0.9251 - acc: 0.7433 - val_loss: nan - val_acc: 0.8809
Epoch 2/10
110288/110288 [==============================] - 43s 387us/step - loss: 0.2507 - acc: 0.9203 - val_loss: nan - val_acc: 0.9442
Epoch 3/10
110288/110288 [==============================] - 42s 385us/step - loss: 0.1498 - acc: 0.9510 - val_loss: nan - val_acc: 0.9567
Epoch 4/10
110288/110288 [==============================] - 43s 390us/step - loss: 0.1089 - acc: 0.9645 - val_loss: nan - val_acc: 0.9659
Epoch 5/10
110288/110288 [==============================] - 43s 386us/step - loss: 0.0827 - acc: 0.9730 - val_loss: nan - val_acc: 0.9738
Epoch 6/10
110288/110288 [==============================] - 42s 384us/step - loss: 0.0688 - acc: 0.9779 - val_loss: nan - val_acc: 0.9756
Epoch 7/10
110288/110288 [==============================] - 42s 385us/step - loss: 0.0588 - acc: 0.9812 - val_loss: nan -

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

**Answer:** To my surprise, creating a train,test and validation split did not change the end results. The accuracy for both the training and validation set had no significant impact.

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
tmp_x=pad(preproc_english_sentences,max_french_sequence_length)

x_train,x_test,y_train,y_test=train_test_split(tmp_x,preproc_french_sentences,train_size=0.8,random_state=54)
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,train_size=0.5,random_state=54)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_val.shape,y_val.shape)

(110288, 21) (110288, 21, 1)
(13786, 21) (13786, 21, 1)
(13787, 21) (13787, 21, 1)


C:\Users\ndrs\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [77]:
def evaluate_pred(y_true,y_pred):
    pred_list=[[[np.argmax(x)] for x in prediction] for prediction in y_pred]
    acc_list=[accuracy_score(y_true[i],pred_list[i])for i in range(len(y_true))]
    print('Mean Accuracy',np.mean(acc_list))

In [78]:
#Simple Model
tmp_x_train=x_train.reshape(-1,x_train.shape[1],1)
tmp_x_test=x_test.reshape(-1,x_test.shape[1],1)
tmp_x_val=x_val.reshape(-1,x_val.shape[1],1)

simple_rnn=simple_model(tmp_x_train.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
simple_rnn.fit(tmp_x_train,y_train,batch_size=32,epochs=10,validation_data=(tmp_x_val,y_val))
predictions=simple_rnn.predict(tmp_x_test)
evaluate_pred(y_test,predictions)

Tensor("input_42:0", shape=(?, 21, 1), dtype=float32)
Train on 110288 samples, validate on 13787 samples
Epoch 1/10
110288/110288 [==============================] - 24s 218us/step - loss: 1.2894 - acc: 0.6366 - val_loss: nan - val_acc: 0.6681
Epoch 2/10
110288/110288 [==============================] - 20s 183us/step - loss: 0.9628 - acc: 0.6949 - val_loss: nan - val_acc: 0.7196
Epoch 3/10
110288/110288 [==============================] - 20s 183us/step - loss: 0.8304 - acc: 0.7326 - val_loss: nan - val_acc: 0.7359
Epoch 4/10
110288/110288 [==============================] - 22s 199us/step - loss: 0.7618 - acc: 0.7503 - val_loss: nan - val_acc: 0.7576
Epoch 5/10
110288/110288 [==============================] - 20s 184us/step - loss: 0.7140 - acc: 0.7645 - val_loss: nan - val_acc: 0.7653
Epoch 6/10
110288/110288 [==============================] - 20s 183us/step - loss: 0.6790 - acc: 0.7748 - val_loss: nan - val_acc: 0.7809
Epoch 7/10
110288/110288 [==============================] - 21s 190

In [79]:
embed_rnn=embed_model(x_train.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
embed_rnn.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_val,y_val))
predictions=embed_rnn.predict(x_test)
evaluate_pred(y_test,predictions)

Tensor("input_43:0", shape=(?, 21), dtype=float32)
Train on 110288 samples, validate on 13787 samples
Epoch 1/10
110288/110288 [==============================] - 23s 209us/step - loss: 0.8114 - acc: 0.7912 - val_loss: nan - val_acc: 0.8885
Epoch 2/10
110288/110288 [==============================] - 22s 199us/step - loss: 0.2799 - acc: 0.9073 - val_loss: nan - val_acc: 0.9168
Epoch 3/10
110288/110288 [==============================] - 22s 201us/step - loss: 0.2275 - acc: 0.9217 - val_loss: nan - val_acc: 0.9199
Epoch 4/10
110288/110288 [==============================] - 22s 196us/step - loss: 0.2028 - acc: 0.9288 - val_loss: nan - val_acc: 0.9298
Epoch 5/10
110288/110288 [==============================] - 25s 224us/step - loss: 0.1908 - acc: 0.9323 - val_loss: nan - val_acc: 0.9310
Epoch 6/10
110288/110288 [==============================] - 24s 218us/step - loss: 0.1816 - acc: 0.9346 - val_loss: nan - val_acc: 0.9304
Epoch 7/10
110288/110288 [==============================] - 24s 219us/

In [80]:
bd_rnn=bd_model(tmp_x_train.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
bd_rnn.fit(tmp_x_train,y_train,batch_size=32,epochs=10,validation_data=(tmp_x_val,y_val))
predictions=bd_rnn.predict(tmp_x_test)
evaluate_pred(y_test,predictions)

Train on 110288 samples, validate on 13787 samples
Epoch 1/10
110288/110288 [==============================] - 32s 286us/step - loss: 1.1390 - acc: 0.6652 - val_loss: nan - val_acc: 0.6929
Epoch 2/10
110288/110288 [==============================] - 30s 270us/step - loss: 0.8501 - acc: 0.7208 - val_loss: nan - val_acc: 0.7164
Epoch 3/10
110288/110288 [==============================] - 28s 257us/step - loss: 0.7194 - acc: 0.7555 - val_loss: nan - val_acc: 0.7693
Epoch 4/10
110288/110288 [==============================] - 28s 256us/step - loss: 0.6426 - acc: 0.7782 - val_loss: nan - val_acc: 0.7933
Epoch 5/10
110288/110288 [==============================] - 28s 255us/step - loss: 0.5907 - acc: 0.7969 - val_loss: nan - val_acc: 0.8031
Epoch 6/10
110288/110288 [==============================] - 28s 255us/step - loss: 0.5391 - acc: 0.8151 - val_loss: nan - val_acc: 0.8222
Epoch 7/10
110288/110288 [==============================] - 28s 255us/step - loss: 0.5034 - acc: 0.8272 - val_loss: nan -

In [81]:
encdec_rnn=encdec_model(tmp_x_train.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
encdec_rnn.fit(tmp_x_train,y_train,batch_size=32,epochs=10,validation_data=(tmp_x_val,y_val))
predictions=encdec_rnn.predict(tmp_x_test)
evaluate_pred(y_test,predictions)

Train on 110288 samples, validate on 13787 samples
Epoch 1/10
110288/110288 [==============================] - 32s 292us/step - loss: 1.4578 - acc: 0.6060 - val_loss: nan - val_acc: 0.6419
Epoch 2/10
110288/110288 [==============================] - 29s 265us/step - loss: 1.0604 - acc: 0.6700 - val_loss: nan - val_acc: 0.6968
Epoch 3/10
110288/110288 [==============================] - 29s 267us/step - loss: 0.9284 - acc: 0.7030 - val_loss: nan - val_acc: 0.7267
Epoch 4/10
110288/110288 [==============================] - 30s 268us/step - loss: 0.8299 - acc: 0.7326 - val_loss: nan - val_acc: 0.7401
Epoch 5/10
110288/110288 [==============================] - 29s 259us/step - loss: 0.7590 - acc: 0.7535 - val_loss: nan - val_acc: 0.7668
Epoch 6/10
110288/110288 [==============================] - 29s 259us/step - loss: 0.7146 - acc: 0.7631 - val_loss: nan - val_acc: 0.7750
Epoch 7/10
110288/110288 [==============================] - 28s 257us/step - loss: 0.6664 - acc: 0.7739 - val_loss: nan -

In [82]:
final_rnn=model_final(x_train.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
final_rnn.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_val,y_val))
predictions=final_rnn.predict(x_test)
evaluate_pred(y_test,predictions)

Train on 110288 samples, validate on 13787 samples
Epoch 1/10
110288/110288 [==============================] - 50s 451us/step - loss: 0.9365 - acc: 0.7422 - val_loss: nan - val_acc: 0.8730
Epoch 2/10
110288/110288 [==============================] - 46s 420us/step - loss: 0.2746 - acc: 0.9138 - val_loss: nan - val_acc: 0.9381
Epoch 3/10
110288/110288 [==============================] - 46s 416us/step - loss: 0.1612 - acc: 0.9468 - val_loss: nan - val_acc: 0.9542
Epoch 4/10
110288/110288 [==============================] - 46s 420us/step - loss: 0.1200 - acc: 0.9604 - val_loss: nan - val_acc: 0.9616
Epoch 5/10
110288/110288 [==============================] - 46s 419us/step - loss: 0.0953 - acc: 0.9688 - val_loss: nan - val_acc: 0.9682
Epoch 6/10
110288/110288 [==============================] - 46s 419us/step - loss: 0.0773 - acc: 0.9748 - val_loss: nan - val_acc: 0.9738
Epoch 7/10
110288/110288 [==============================] - 46s 418us/step - loss: 0.0657 - acc: 0.9785 - val_loss: nan -

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [83]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 368062 bytes to machine_translation.html']